# DATA 620 - Assignment 3 Alt

by: Jeremy OBrien and Mael Illien

Load a graph database of your choosing from a text file or other source. If you take a large network dataset from the web (such as from Stanford Large Network Dataset Collection (https://snap.stanford.edu/data/)), please feel free at this point to load just a small subset of the nodes and edges.

Create basic analysis on the graph, including the graph’s diameter, and at least one other metric of your choosing. You may either code the functions by hand (to build your intuition and insight), or use functions in an existing package.

Use a visualization tool of your choice (Neo4j, Gephi, etc.) to display information. Please record a short video (~ 5 minutes), and submit a link to the video in advance of our meet-up.

In [1]:
import os
import random
import pandas as pd
import networkx as nx

Data sourced from: https://snap.stanford.edu/data/soc-sign-bitcoin-otc.html

In [2]:
# Import data
url = 'https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/soc-sign-bitcoinotc.csv'
ratings = pd.read_csv(url, names=['SOURCE','TARGET','RATING','TIMESTAMP'] )
ratings

,SOURCE,TARGET,RATING,TIMESTAMP
0,6,2,4,1.289242e+09
1,6,5,2,1.289242e+09
2,1,15,1,1.289243e+09
3,4,3,7,1.289245e+09
4,13,16,8,1.289254e+09
...,...,...,...,...
35587,4499,1810,1,1.453612e+09
35588,2731,3901,5,1.453679e+09
35589,2731,4897,5,1.453679e+09
35590,13,1128,1,1.453680e+09


#### Load data into a graph

In [3]:
data = open('soc-sign-bitcoinotc.csv', 'r')
graphtype = nx.DiGraph()
graph = nx.parse_edgelist(data, 
                      delimiter=',', 
                      create_using=graphtype,
                      nodetype=int, 
                      data=(('weight', float),('time', float)))

In [4]:
len(graph.nodes)

5881

In [5]:
len(graph.edges)

35592

In [ ]:
# Expand on this graph as needed

### Cypher Query Generation 

In [6]:
# Randomly sample 100 users
unique_users = set(ratings['SOURCE']) # Identify unique users
random.seed(620)
ratings_subset = ratings[ratings['SOURCE'].isin(random.sample(unique_users, 50))]

source_nodes = set(ratings_subset['SOURCE'])
target_nodes = set(ratings_subset['TARGET'])
all_nodes = source_nodes.union(target_nodes) 



### Method 1: Write queries to file and copy and paste into Neo4j. Inefficient

In [ ]:
# Create nodes file

# Delete the file if it already exists
if os.path.exists("create_nodes.txt"):
      os.remove("create_nodes.txt")

# Write the queries to file
with open("create_nodes.txt", "a") as f:     
   
    # Query format for node creation: "CREATE (:User {id: '6'})"
    for node in all_nodes:
        query = "CREATE (:User {id: '%d'}); \n" % (node)
        f.write(query)

In [7]:
len(all_nodes)

523

In [8]:
ratings_subset

,SOURCE,TARGET,RATING,TIMESTAMP
90,41,36,2,1.292206e+09
127,41,75,1,1.293584e+09
139,41,60,1,1.293820e+09
148,41,4,1,1.294081e+09
159,41,83,1,1.294263e+09
...,...,...,...,...
35291,5972,13,1,1.437803e+09
35304,41,5974,2,1.438383e+09
35432,3719,1272,-10,1.442418e+09
35433,41,4291,2,1.442787e+09


In [9]:
ratings_subset = ratings_subset.reset_index()

In [10]:
ratings_subset

,index,SOURCE,TARGET,RATING,TIMESTAMP
0,90,41,36,2,1.292206e+09
1,127,41,75,1,1.293584e+09
2,139,41,60,1,1.293820e+09
3,148,41,4,1,1.294081e+09
4,159,41,83,1,1.294263e+09
...,...,...,...,...,...
581,35291,5972,13,1,1.437803e+09
582,35304,41,5974,2,1.438383e+09
583,35432,3719,1272,-10,1.442418e+09
584,35433,41,4291,2,1.442787e+09


In [12]:
# Create relationships query file

# Delete the file if it already exists
import os
if os.path.exists("create_relationships.txt"):
      os.remove("create_relationships.txt")

# Query format for relationship creation
# "MATCH (u1:User {id: '6'}),(u2:User {id: '2'}) WITH u1,u2 CREATE (u1)-[:TRUSTS {rating: 4}]->(u2) "
        
# Write the queries to file
with open("create_relationships.txt", "a") as f: 
    for i in ratings_subset.index:
        src = ratings_subset['SOURCE'][i]
        tar = ratings_subset['TARGET'][i]
        rating = ratings_subset['RATING'][i]
        query = "MATCH (u1:User {id: '%s'}),(u2:User {id: '%s'}) WITH u1,u2 CREATE (u1)-[:TRUSTS {rating: %s}]->(u2) ; \n" % (src, tar, rating) 
        f.write(query)
        

Once the queries are written to write, they can be imported in Neo4j Browser via copy and paste. But this is not an efficient way to load.

### Sample Queries:

In [14]:
# Return all the nodes
"MATCH (n) RETURN n"

'MATCH (n) RETURN n'

In [15]:
# Delete everything
"MATCH (n) DETACH DELETE n"

'MATCH (n) DETACH DELETE n'

#### Method 2: Write the dataframe to csv and parse csv into Neo4j

In [16]:
df = pd.DataFrame(all_nodes)
df.columns = ['id']
df.to_csv('nodes.csv')

In [17]:
df = ratings_subset[['SOURCE','TARGET','RATING']]
df.to_csv('edges.csv')

In [18]:
# Create nodes
"""LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/nodes.csv" AS csvLine
CREATE (u:User {id: (csvLine.id)})"""

'LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/nodes.csv" AS csvLine\nCREATE (u:User {id: (csvLine.id)})'

In [19]:
# Create relationships
"""LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/edges.csv" AS csvLine
MATCH (u1:User {id: csvLine.SOURCE})
MATCH (u2:User {id: csvLine.TARGET})
CREATE (u1)-[:TRUSTS {rating: csvLine.RATING}]->(u2)"""

'LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/edges.csv" AS csvLine\nMATCH (u1:User {id: csvLine.SOURCE})\nMATCH (u2:User {id: csvLine.TARGET})\nCREATE (u1)-[:TRUSTS {rating: csvLine.RATING}]->(u2)'